In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in c:\users\bandana\anaconda3\lib\site-packages (0.0)


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
#Dependencies

import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [5]:
df.koi_disposition.value_counts()

FALSE POSITIVE    3504
CONFIRMED         1800
CANDIDATE         1687
Name: koi_disposition, dtype: int64

In [6]:
# look at confirmed as true and false positives as false (1 and 0) and remove all candidate planets.
df = df.loc[df["koi_disposition"] != "CANDIDATE"]

df.koi_disposition.value_counts()

FALSE POSITIVE    3504
CONFIRMED         1800
Name: koi_disposition, dtype: int64

In [7]:
df['koi_disposition'] = df["koi_disposition"].apply(lambda x: 1 if(x=="CONFIRMED") else 0)
y = df['koi_disposition']
df.koi_disposition.value_counts()


0    3504
1    1800
Name: koi_disposition, dtype: int64

In [8]:
# There are three different categories the planets can be classified.
# Let's start with all features and then remove features that have little weight.

#  Avoid linear regression, this is a classification problem.
# Start with logistic classification.
df.columns

Index(['koi_disposition', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_period_err1', 'koi_period_err2',
       'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact',
       'koi_impact_err1', 'koi_impact_err2', 'koi_duration',
       'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_depth_err1',
       'koi_depth_err2', 'koi_prad', 'koi_prad_err1', 'koi_prad_err2',
       'koi_teq', 'koi_insol', 'koi_insol_err1', 'koi_insol_err2',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_steff_err1',
       'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2',
       'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec',
       'koi_kepmag'],
      dtype='object')

# Select your features (columns)

In [9]:
# Set features. This will also be used as your x values.
#selected_features = df[['names', 'of', 'selected', 'features', 'here']]

selected_features = df[['koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration', 'koi_depth', 'koi_prad', 'koi_teq', 'koi_insol', 'koi_model_snr',
                        'koi_tce_plnt_num', 'koi_steff', 'koi_slogg', 'koi_srad', 'ra', 'dec', 'koi_kepmag']]


# Create a Train Test Split

Use `koi_disposition` for the y values

In [10]:
from sklearn.model_selection 

import train_test_split

X = selected_features

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size=0.8, random_state=42)

SyntaxError: invalid syntax (<ipython-input-10-7d63964ccc2b>, line 1)

In [ ]:
X_train.head()

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [ ]:
# Scale your data

# Setup MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

# Set initial sizing of minmaxscaler using training data
X_scaler = MinMaxScaler().fit(X_train)

# As y is just 1 or zero, so no need to scale it!
# y_scaler = MinMaxScaler().fit(y_train)

In [ ]:
# Use MinMaxScaler to transform the data in the training and test data sets
# So that it is not weighted improperly

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
# y_train_scaled = y_scaler.transform(y_train)
# y_test_scaled = y_scaler.transform(y_test)

# Train the Model



In [ ]:
#first, use logistic regression in this notebook!
from sklearn.linear_model import LogisticRegression
model2 = LogisticRegression()
model2

In [ ]:
#create the model using the training data!
model2.fit(X_train_scaled, y_train)

In [ ]:
print(f"Training Data Score: {model2.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model2.score(X_test_scaled, y_test)}")

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [ ]:
# Create the GridSearchCV model

from sklearn.model_selection import GridSearchCV

param_grid = {'C': [1, 5, 10, 50, 100, 200, 500],
              'max_iter': [100, 200, 300],
              'tol': [.0001, 0.001, 0.005]}

grid2 = GridSearchCV(model2, param_grid, verbose=3)

In [ ]:
# Train the model with GridSearch
grid2.fit(X_train_scaled, y_train)


In [ ]:
print(grid2.best_params_)
print(grid2.best_score_)

In [ ]:
#compare with the test data!
predictions = grid2.predict(X_test_scaled)
print(sum(predictions), sum(y_test))

total = 0

for i in range(len(predictions)):
    if(predictions[i] == y_test.array[i]):
        total +=1 
        
print(total/len(predictions))

In [ ]:
#check classification report for how the tuned model did!
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions,
                            target_names=["Confirmed", "False Positive"]))

# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'Bandana_model.sav'
joblib.dump(grid2, filename)